In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\prati\\Desktop\\Project\\English-to-Hindi-Translator'

In [5]:
#Update in entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source: str
    local_data_file: Path
    

In [6]:
from src.translator.constants import *
from src.translator.utils.common import read_yaml, create_directories

In [7]:
#Update configuration.py
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source=config.source,
            local_data_file=config.local_data_file,
        )

        return data_ingestion_config

In [8]:
import os
import datasets
import pandas as pd
from src.translator.logging import logger
from src.translator.utils.common import get_size

c:\Users\prati\Desktop\Project\English-to-Hindi-Translator\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#Update data_ingestion.py
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            logger.info("Downloading data")
            dataset = datasets.load_dataset(f'{self.config.source}')

            # Step 2: Convert the dataset to a Pandas DataFrame
            df_train = pd.DataFrame(dataset["train"])
            df_test = pd.DataFrame(dataset["test"])
            df_validation= pd.DataFrame(dataset["validation"])
            
            # Save the DataFrame to a CSV file in the desired folder
            df_train.to_csv(self.config.root_dir + "/data/train.csv", index=False)
            df_validation.to_csv(self.config.root_dir + "/data/validation.csv", index=False)
            df_test.to_csv(self.config.root_dir + "/data/test.csv", index=False)
            
            logger.info("Dataset ingestion succesful")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e

[2023-08-10 11:14:11,629: INFO: common: yaml file: config\config.yaml loaded successfully]


[2023-08-10 11:14:11,660: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-10 11:14:11,665: INFO: common: created directory at: artifacts]
[2023-08-10 11:14:11,669: INFO: common: created directory at: artifacts/data_ingestion]
[2023-08-10 11:14:11,673: INFO: 11085498: File already exists of size: ~ 463441 KB]
